# LDA Model Training

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Data

In [35]:
data_file_path='DATA.csv'
data=pd.read_csv(data_file_path)
data

,PeakID,Distance to TSS,motif_string
0,seq_40800,-81,"motif_149,motif_162,motif_166,motif_166,motif_..."
1,seq_10314,-228,motif_144
2,seq_18866,9616,"motif_104,motif_126,motif_126,motif_147,motif_..."
3,seq_45348,5770,"motif_120,motif_121,motif_128,motif_128,motif_..."
4,seq_2616,-12540,"motif_100,motif_103,motif_126,motif_126,motif_..."
...,...,...,...
60095,seq_5507,-127293,"motif_137,motif_137,motif_13,motif_174,motif_1..."
60096,seq_11070,37721,"motif_132,motif_132,motif_132,motif_142,motif_..."
60097,seq_17945,88589,"motif_161,motif_162,motif_162,motif_174,motif_..."
60098,seq_29137,93216,"motif_104,motif_105,motif_117,motif_117,motif_..."


In [36]:
print(data[data['motif_string'].isna()])
data.dropna(subset=['motif_string'],inplace=True)
data['motif_list']=data['motif_string'].apply(lambda x:x.split(','))
data

          PeakID  Distance to TSS motif_string
18840  seq_57232             -180          NaN
23892  seq_20216             -147          NaN
29667  seq_31231              483          NaN
46832   seq_8465             -341          NaN
47219  seq_46451             4737          NaN


,PeakID,Distance to TSS,motif_string,motif_list
0,seq_40800,-81,"motif_149,motif_162,motif_166,motif_166,motif_...","[motif_149, motif_162, motif_166, motif_166, m..."
1,seq_10314,-228,motif_144,[motif_144]
2,seq_18866,9616,"motif_104,motif_126,motif_126,motif_147,motif_...","[motif_104, motif_126, motif_126, motif_147, m..."
3,seq_45348,5770,"motif_120,motif_121,motif_128,motif_128,motif_...","[motif_120, motif_121, motif_128, motif_128, m..."
4,seq_2616,-12540,"motif_100,motif_103,motif_126,motif_126,motif_...","[motif_100, motif_103, motif_126, motif_126, m..."
...,...,...,...,...
60095,seq_5507,-127293,"motif_137,motif_137,motif_13,motif_174,motif_1...","[motif_137, motif_137, motif_13, motif_174, mo..."
60096,seq_11070,37721,"motif_132,motif_132,motif_132,motif_142,motif_...","[motif_132, motif_132, motif_132, motif_142, m..."
60097,seq_17945,88589,"motif_161,motif_162,motif_162,motif_174,motif_...","[motif_161, motif_162, motif_162, motif_174, m..."
60098,seq_29137,93216,"motif_104,motif_105,motif_117,motif_117,motif_...","[motif_104, motif_105, motif_117, motif_117, m..."


In [37]:
from gensim.corpora import Dictionary

docs=data['motif_list'].values
dictionary=Dictionary(docs)

print('Motif-Index to Motif-Name Mapping:')
for i,v in dictionary.items():
    print(f'{i} - {v}')
    if i==10:
        break
    

Motif-Index to Motif-Name Mapping:
0 - motif_149
1 - motif_162
2 - motif_166
3 - motif_174
4 - motif_199
5 - motif_230
6 - motif_248
7 - motif_295
8 - motif_60
9 - motif_66
10 - motif_98


In [38]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('BOW (Sequence-0):')
docs[0],corpus[0]

BOW (Sequence-0):


(['motif_149',
  'motif_162',
  'motif_166',
  'motif_166',
  'motif_174',
  'motif_199',
  'motif_230',
  'motif_248',
  'motif_295',
  'motif_295',
  'motif_60',
  'motif_60',
  'motif_66',
  'motif_98'],
 [(0, 1),
  (1, 1),
  (2, 2),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 2),
  (9, 1),
  (10, 1)])

## Training

In [39]:
# #setup logging for trainging metrics 
# import logging
# logging.basicConfig(filename='test_output/model_callbacks.log', filemode='w',
#                     format="%(asctime)s:%(levelname)s:%(message)s",
#                     level=logging.NOTSET)

# from gensim.models.callbacks import Callback,PerplexityMetric, ConvergenceMetric, CoherenceMetric
# perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
# convergence_logger = ConvergenceMetric(logger='shell')
# # coherence_cv_logger = CoherenceMetric(corpus=corpus, coherence = 'c_v', texts = docs)

# %%time
# from gensim.models import LdaModel,LdaMulticore

# #HYPERPARAMETERS
# #passes = epochs
# temp = dictionary[0]
# id2word = dictionary.id2token
# lda = LdaModel(corpus, id2word=id2word, alpha='auto',eval_every = 1,\
#                eta='auto',num_topics=3, iterations=5, passes = 10,
#               callbacks=[perplexity_logger,convergence_logger])

# lda.print_topics()

# %%time
# from gensim.models import LdaModel,LdaMulticore

# id2word = dictionary.id2token
# lda = LdaMulticore(corpus, id2word=id2word,eval_every = 1,\
#                eta='auto',num_topics=3, iterations=500, passes = 100)

# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()
# pyLDAvis.gensim.prepare(lda, corpus, dictionary)

In [40]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

In [48]:
%%time
from gensim.models import LdaModel,LdaMulticore

temp = dictionary[0]
id2word = dictionary.id2token
lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
               eta='auto',num_topics=3, iterations=1000, passes = 5,
              minimum_probability=0.0)

CPU times: user 1min 48s, sys: 78 ms, total: 1min 48s
Wall time: 1min 48s


In [62]:
def getTopicDistribution(predictions,ntopics):
    topic_count_dict={i:0 for i in range(ntopics)}
    ndocs=len(predictions)
    print(ndocs)
    for pred in predictions:
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        topic_count_dict[top_topic]+=1
    topic_dist_dict={k:v/ndocs for k,v in topic_count_dict.items()}
    return topic_dist_dict

def likelihoodMetric(predictions,ntopics):
    likelihood=0
#     P_T=model.alpha
#     predictions=lda.get_document_topics(corpus,minimum_probability=0.0)
    P_T=getTopicDistribution(predictions,model.num_topics)
    print(P_T)
    for pred in predictions:
        P_Xi_M=0
        for topic_no,P_Xi_T in pred:
            P_Xi_M+=P_Xi_T*P_T[topic_no]
        likelihood+=np.log10(P_Xi_M)
    print(likelihood)
    return likelihood

In [57]:
from collections import Counter

def findTopMotifs(predictions,ntopics,data,ntop=5,outdir=None):
    pred_topic=[]
#     ntopics=model.num_topics
    for pred in predictions:
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        pred_topic.append(top_topic)
    _data=data.copy()
    _data['pred_topic']=pred_topic      
    gb=_data[['motif_string','pred_topic']].groupby('pred_topic').\
    agg(lambda x: ','.join(x))
    gb['top_motif']=gb['motif_string'].\
    apply(lambda x:Counter(x.split(',')).most_common(ntop))
    gb.reset_index(inplace=True)
    gb=gb[['pred_topic','top_motif']]
    if outdir is not None:
        gb.to_csv(f'{outdir}/top{ntop}_motifs_topics_{ntopics}.csv',index=False)
    print(gb)
    return gb
# findTopMotifs(lda,corpus,data)

In [58]:
def getAvgTssDist(predictions,ntopics,data,outdir=None):
    pred_topic=[]
#     ntopics=model.num_topics
    for pred in predictions:
        top_topic=sorted(pred,key=lambda x:-x[1])[0][0]
        pred_topic.append(top_topic)
    _data=data.copy()
    _data['pred_topic']=pred_topic
    gb=_data[['Distance to TSS','pred_topic']].groupby('pred_topic').mean()
    gb.reset_index(inplace=True)
    if outdir is not None:
        gb.to_csv(f'{outdir}/avg_tss_dist_topics_{ntopics}.csv',index=False)
    print(gb)
    return gb
# getAvgTssDist(lda,corpus,data)

In [63]:
%%time
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from gensim.models import LdaModel,LdaMulticore

outdir='model_output'
eval_dict={'num_topics':[],'likelihood':[]}
temp = dictionary[0]
id2word = dictionary.id2token
for ntopics in range(2,10):
    print('Num of Topics = '+str(ntopics))
    lda = LdaModel(corpus, id2word=id2word, alpha='auto',chunksize=10000,
                   eta='auto',num_topics=ntopics, iterations=1000, passes = 5,
                  minimum_probability=0.0)
    
    predictions=lda.get_document_topics(corpus,minimum_probability=0.0)
    print('Finding likelihood...')
    likelihood=likelihoodMetric(predictions,ntopics)
    print('Finding Top Motifs...')
    findTopMotifs(predictions,ntopics,data,outdir=outdir)
    print('Findng avg. distance from TSS per topic...')
    getAvgTssDist(predictions,ntopics,data,outdir=outdir)
    eval_dict['num_topics'].append(ntopics)
    eval_dict['likelihood'].append(likelihood)
eval_df=pd.DataFrame(eval_dict)
eval_df.to_csv(f'{outdir}/likelihood.csv',index=False)
eval_df

Num of Topics = 2
Finding likelihood...
60095
{0: 0.47805973874698393, 1: 0.5219402612530161}
-18068.728802235655
Num of Topics = 3
Finding likelihood...
60095
{0: 0.4924203344704218, 1: 0.3915134370579915, 2: 0.11606622847158665}
-24259.491999566286
Num of Topics = 4
Finding likelihood...
60095
{0: 0.08749479990015809, 1: 0.19722106664447958, 2: 0.334320658956652, 3: 0.3809634744987104}
-32063.50715062038
Num of Topics = 5
Finding likelihood...
60095
{0: 0.26448123803977036, 1: 0.1906315001248024, 2: 0.10553290623179964, 3: 0.12280555786671105, 4: 0.3165487977369166}
-39058.58057453927
Num of Topics = 6
Finding likelihood...
60095
{0: 0.1115733422081704, 1: 0.2786255096097845, 2: 0.17810133954571927, 3: 0.07305100257924953, 4: 0.30373575172643313, 5: 0.05491305433064315}
-40719.22278258542
Num of Topics = 7
Finding likelihood...
60095
{0: 0.2614027789333555, 1: 0.1528746151926117, 2: 0.036275896497212745, 3: 0.05637740244612697, 4: 0.16087860886929028, 5: 0.043963724103502785, 6: 0.28

,num_topics,likelihood
0,2,-18068.728802
1,3,-24259.492000
2,4,-32063.507151
3,5,-39058.580575
4,6,-40719.222783
5,7,-42448.172350
6,8,-47916.775584
7,9,-52662.875743
